# This code is the first preprocessing step for the data, it considers only arrival bugs and  count those bugs that arrived on the same day

In [115]:
!pip install openpyxl
!pip install xlsxwriter

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: /usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip


In [2]:
import pandas as pd
from collections import defaultdict
import numpy as np
from sortedcontainers import SortedDict
import xlsxwriter
import collections 

dbNames = ['Tizen', 'Cyanogenmod','Nemo', 'Mer']


In [5]:
for dbName in dbNames:
    bugArrival = None

    with pd.ExcelFile(f'./{dbName}_bugs.xlsx') as reader:
        bugArrival = pd.read_excel(reader, sheet_name='Bug Arrival')    
    
    mydict = defaultdict(int)


    if dbName != 'Mer' and dbName != 'Nemo': 
        bugArrival['Created'] = pd.to_datetime(bugArrival['Created'], errors='coerce').dt.strftime('%Y-%m-%d').astype(str)
        bugArrival['Created.1'] = pd.to_datetime(bugArrival['Created.1'], errors='coerce').dt.strftime('%Y-%m-%d').astype(str)
        for date in bugArrival['Created.1']:
            if date == 'nan':
                break
            mydict[date] += 1

        for date in bugArrival['Created']:
            if date == 'nan':
                break
            mydict[date] += 1
            
    else:
        bugArrival['Opened'] = pd.to_datetime(bugArrival['Opened'], errors='coerce').dt.strftime('%Y-%m-%d').astype(str)

        for date in bugArrival['Opened']:
            if date == 'nan':
                break
            mydict[date] += 1
    


    sortedlist = SortedDict(mydict)
    
    
    dateList = pd.date_range(start= sortedlist.keys()[0],end=sortedlist.keys()[len(sortedlist)-1])

    workbook = xlsxwriter.Workbook(f'countedSRs{dbName}Bugs.xlsx')
    worksheet = workbook.add_worksheet()
    row = 0

    # iterating through content list
    for date in dateList:
        d = date.strftime('%Y-%m-%d')
        worksheet.write(row, 0, row+1)
        if d not in sortedlist.keys():
            worksheet.write(row, 1, 0)
        else:
            worksheet.write(row, 1, sortedlist[d])
        row += 1

    workbook.close()